# Berkeley Housing Pipeline - Permitting, Construction, Economic and Neighborhood Outcomes 
- January 4, 2026

## [Measuring Actual Outcomes of new housing policies](https://www.eatingpolicy.com/p/outcomes-reviews-realigning-legislative?utm_campaign=post)

This **Master Analysis Notebook** will be refined into a set of Jupyter Notebooks with independent notebooks for each step: data acquisition and cleaning; statistical analysis; visualization and mapping; report generation. 

## This set of computational notebooks should complement and extend the [City of Berkeley's Annual Housing Element Progress Report](https://berkeleyca.gov/sites/default/files/documents/2025-03-28%20%20%20Housing%20Element%20and%20General%20Plan%20Annual%20Progress%20Reports.pdf) to the State of California HCD


### This is built as a template that may be cloned from Github, then completely modified and run locally on your machine, or at Google Colab.

For Berkeley High classes, UC Berkeley classes, or Berkeleyside, Berkeley Scanner, Daily Cal or other data journalism sites, the SQL data repositories may be used with Datasette. Claude can generate SQL queries to be used interactively with Datasette to create interactive maps, statistical analyses, and a wide variety of time series analyses.

As the City of Berkeley builds better API interfaces to the City Open.Gov databases of permits for Planning, Building, Design Review, and Certificates of Occupancy, the richness of analysis will improve.  We will add all Berkeley Business Licenses, all publicly available energy and water use data, and displacement and affordabiity analyses. 

We will analyze actual outcomes of Berkeley and State of California policies, laws, and regulations. 
for Berkeley businesses and residences


This is a complete workflow from data → analysis → visualization → database
- started Jan 3 - Sat- 10:27 am
- Jan 4 - 2:24

## Setup

In [2]:
# CELL: Setup for Google Colab (auto-detects, won't run in local Jupyter)

import sys
import os

# Check if running in Google Colab
if 'google.colab' in sys.modules:
    print("🌐 Running in Google Colab - downloading data from GitHub...")
    
    # Download the data file from GitHub
    import urllib.request
    
    data_url = "https://raw.githubusercontent.com/blockXblock/berkeley-housing-analysis/main/housing_projects_final_complete.csv"
    
    urllib.request.urlretrieve(data_url, "housing_projects_final_complete.csv")
    
    print("✅ Data downloaded!")
    
    # Set data directory for Colab
    data_dir = "/content"
else:
    print("💻 Running locally")
    data_dir = "/Users/johngage/berkeley-data"

💻 Running locally


In [3]:
import pandas as pd
import sqlite3
import folium
from datetime import datetime
import os

print('✅ Imports successful')
print(f'Working directory: {os.getcwd()}')

✅ Imports successful
Working directory: /Users/johngage/berkeley-data/notebooks


## 🔄 DATA REFRESH (Optional)

**Choose your approach:**
- **Level 1:** Quick refresh using existing data *(recommended, 30 seconds)*
- **Level 2:** Complete rebuild from Berkeley Open Data API *(advanced, 5-10 minutes)*

Skip to **Section 1** if you want to use existing data as-is.

### Level 1: Quick Data Refresh

Check if data files exist and are recent. Optionally re-geocode any missing addresses.

In [4]:
# LEVEL 1: Quick refresh
import pandas as pd
import os
from datetime import datetime

data_dir = '/Users/johngage/berkeley-data'

# Check data files
print('📊 DATA FILE STATUS:')
print('='*70)

files_to_check = [
    'housing_projects_final_complete.csv',
    'alameda_lookup_complete.csv',
    'berkeley_addresses_with_fields.csv'
]

for filename in files_to_check:
    filepath = f'{data_dir}/{filename}'
    if os.path.exists(filepath):
        stat = os.stat(filepath)
        size_mb = stat.st_size / (1024 * 1024)
        modified = datetime.fromtimestamp(stat.st_mtime)
        days_old = (datetime.now() - modified).days
        
        print(f'\n✅ {filename}')
        print(f'   Size: {size_mb:.1f} MB')
        print(f'   Modified: {modified.strftime("%Y-%m-%d")} ({days_old} days ago)')
    else:
        print(f'\n❌ {filename} - NOT FOUND')
        print(f'   → Need to run Level 2: Complete Rebuild')

# Load and check projects
projects_file = f'{data_dir}/housing_projects_final_complete.csv'
if os.path.exists(projects_file):
    df = pd.read_csv(projects_file)
    geocoded = df['latitude'].notna().sum()
    total = len(df)
    
    print(f'\n📈 PROJECTS DATA:')
    print(f'   Total projects: {total}')
    print(f'   Geocoded: {geocoded} ({geocoded/total*100:.1f}%)')
    print(f'   Missing coordinates: {total - geocoded}')
    
    if geocoded < total:
        print(f'\n⚠️  {total - geocoded} addresses need geocoding')
        print(f'   Run geocoding cell below to fill gaps')
    else:
        print(f'\n✅ All projects geocoded! Data is ready to use.')
else:
    print(f'\n❌ Projects file not found. Run Level 2: Complete Rebuild.')

print('\n' + '='*70)

📊 DATA FILE STATUS:

✅ housing_projects_final_complete.csv
   Size: 0.0 MB
   Modified: 2025-12-20 (15 days ago)

✅ alameda_lookup_complete.csv
   Size: 59.1 MB
   Modified: 2025-12-19 (16 days ago)

✅ berkeley_addresses_with_fields.csv
   Size: 13.0 MB
   Modified: 2025-12-20 (15 days ago)

📈 PROJECTS DATA:
   Total projects: 84
   Geocoded: 84 (100.0%)
   Missing coordinates: 0

✅ All projects geocoded! Data is ready to use.



---

### Level 2: Complete Data Rebuild (Advanced)

⚠️ **Warning:** This fetches fresh data from Berkeley Open Data Portal and rebuilds everything from scratch.

**Time:** 5-10 minutes  
**Requires:** Internet connection, Berkeley Open Data API access

**Instructions:** Uncomment cells below to activate the rebuild process.

In [5]:
# LEVEL 2 - STEP 1: Fetch Berkeley Open Data
# Uncomment the code below to run

print('(Level 2 cells are commented out by default)')
print('Uncomment to activate data rebuild from Berkeley Open Data Portal')

# Placeholder - add Berkeley API code here after finding dataset ID

(Level 2 cells are commented out by default)
Uncomment to activate data rebuild from Berkeley Open Data Portal


---

## ✅ Data Refresh Complete

Proceed to **Section 1: Load Data** below.

## 1. Load Data

Using existing geocoded projects (84 projects, 100% geocoded)

In [6]:
# Load projects (path relative to notebooks directory)
data_dir = '/Users/johngage/berkeley-data'
df = pd.read_csv(f'{data_dir}/housing_projects_final_complete.csv')

print(f'📊 Loaded: {len(df)} projects')
print(f'   With coordinates: {df["latitude"].notna().sum()}')
print(f'   Total units: {df["net_units"].sum():,.0f}')
print(f'   Date range: {df["year"].min():.0f} - {df["year"].max():.0f}')

# Show sample
df.head()

📊 Loaded: 84 projects
   With coordinates: 84
   Total units: 5,283
   Date range: 2020 - 2025


,id,address_display,apn,owner,net_units,new_units,old_units,year,permits,description,status,latitude,longitude,num_permits,project_size_category,slug
0,1,1750 SACRAMENTO St,058 214901904,NaN,739.0,739.0,0.0,2024.0,PLN2024-0010,AB 2011 Ministerial Application. The proposed ...,Under Review,37.874312,-122.282959,1,Mega (100+),1750-sacramento-st
1,2,2276 SHATTUCK Ave,057 202800400,NaN,336.0,336.0,0.0,2024.0,ZP2024-0067,SB330 Preliminary Application: Retaining of th...,In Review,37.867738,-122.268240,1,Mega (100+),2276-shattuck-ave
2,3,2700 SHATTUCK Ave,054 172300100,NaN,276.0,276.0,0.0,2024.0,ZP2024-0058,SB330 project to demolish an existing commerci...,Corrections Pending Applicant,37.859780,-122.267828,1,Mega (100+),2700-shattuck-ave
3,4,1914 FIFTH St,057 210000702,NaN,257.0,257.0,0.0,2020.0,ZP2020-0104,demolish an existing parking lot and portions ...,Under Review,37.868230,-122.299296,1,Mega (100+),1914-fifth-st
4,5,2425 DURANT Ave,055 187800701,NaN,250.0,250.0,19.0,2024.0,"ZP2024-0162, PLN2024-0049",SB-330 PROJECT - Demolition of all existing bu...,Pending Final Action,37.867951,-122.260142,2,Mega (100+),2425-durant-ave


## 2. Summary Statistics

In [7]:
print('📈 SUMMARY STATISTICS')
print('='*70)

# Overall stats
print(f"\nTotal projects: {len(df)}")
print(f"Total units: {df['net_units'].sum():,.0f}")
print(f"Average units per project: {df['net_units'].mean():.1f}")
print(f"Median units per project: {df['net_units'].median():.0f}")
print(f"Largest project: {df['net_units'].max():.0f} units at {df.loc[df['net_units'].idxmax(), 'address_display']}")

# Projects by year
print('\n📅 PROJECTS BY YEAR:')
by_year = df.groupby('year').agg({
    'net_units': ['sum', 'count'],
    'address_display': 'first'
})
by_year.columns = ['total_units', 'num_projects', '_']
by_year = by_year[['num_projects', 'total_units']]
print(by_year.to_string())

# Projects by size category
print('\n📏 PROJECTS BY SIZE:')
size_bins = [0, 20, 50, 100, 200, 1000]
size_labels = ['Small (<20)', 'Small-Medium (20-49)', 'Medium (50-99)', 'Large (100-199)', 'Very Large (200+)']
df['size_category'] = pd.cut(df['net_units'], bins=size_bins, labels=size_labels)
by_size = df.groupby('size_category', observed=True).agg({
    'net_units': ['count', 'sum']
})
by_size.columns = ['num_projects', 'total_units']
print(by_size.to_string())

📈 SUMMARY STATISTICS

Total projects: 84
Total units: 5,283
Average units per project: 62.9
Median units per project: 10
Largest project: 739 units at 1750 SACRAMENTO St

📅 PROJECTS BY YEAR:
        num_projects  total_units
year                             
2020.0             2        258.0
2021.0             3         70.0
2022.0             8        782.0
2023.0             7        388.0
2024.0            32       3420.0
2025.0            32        365.0

📏 PROJECTS BY SIZE:
                      num_projects  total_units
size_category                                  
Small (<20)                     42        173.0
Small-Medium (20-49)             6        213.0
Medium (50-99)                  15       1059.0
Large (100-199)                  8       1085.0
Very Large (200+)                9       2754.0


## 3. Geographic Analysis

In [8]:
# Top streets by development
print('🏘️ TOP STREETS BY HOUSING UNITS:')
print('='*70)

# Extract street name from address
df['street'] = df['address_display'].str.split().str[1:].str.join(' ')

top_streets = df.groupby('street').agg({
    'net_units': 'sum',
    'address_display': 'count'
}).rename(columns={'address_display': 'num_projects'}).sort_values('net_units', ascending=False)

print('\nTop 15 streets:')
print(top_streets.head(15).to_string())

# Geographic distribution
print('\n📍 GEOGRAPHIC COORDINATES:')
print(f"  Latitude range: {df['latitude'].min():.6f} to {df['latitude'].max():.6f}")
print(f"  Longitude range: {df['longitude'].min():.6f} to {df['longitude'].max():.6f}")
print(f"  All coordinates within Berkeley bounds: {((df['latitude'] >= 37.84) & (df['latitude'] <= 37.91) & (df['longitude'] >= -122.32) & (df['longitude'] <= -122.23)).all()}")

🏘️ TOP STREETS BY HOUSING UNITS:

Top 15 streets:
                net_units  num_projects
street                                 
SHATTUCK Ave       1564.0            10
SACRAMENTO St       742.0             2
DURANT Ave          453.0             4
UNIVERSITY Ave      452.0             6
SAN PABLO Ave       437.0             5
FIFTH St            272.0             5
OXFORD St           212.0             1
KITTREDGE St        211.0             2
BANCROFT Way        200.0             3
TELEGRAPH Ave       156.0             3
VIRGINIA St         131.0             1
MILVIA St           113.0             2
LE CONTE Ave         65.0             1
ELLSWORTH St         63.0             1
CENTER St            58.0             1

📍 GEOGRAPHIC COORDINATES:
  Latitude range: 37.851354 to 37.900125
  Longitude range: -122.301952 to -122.250275
  All coordinates within Berkeley bounds: True


## 4. Interactive Map

In [9]:
print('🗺️ Creating interactive map...')

# Create base map
m = folium.Map(
    location=[37.8715, -122.2730],
    zoom_start=13,
    tiles='OpenStreetMap'
)

# Color by project size
def get_color(units):
    if units >= 200:
        return 'red'
    elif units >= 100:
        return 'orange'
    elif units >= 50:
        return 'blue'
    elif units >= 20:
        return 'green'
    else:
        return 'lightblue'

# Add markers for all geocoded projects
for idx, row in df[df['latitude'].notna()].iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=6,
        popup=f"<b>{row['address_display']}</b><br>Units: {row['net_units']}<br>Year: {row['year']:.0f}<br>Status: {row.get('status', 'N/A')}",
        color=get_color(row['net_units']),
        fill=True,
        fillColor=get_color(row['net_units']),
        fillOpacity=0.7
    ).add_to(m)

# Save map to outputs directory
outputs_dir = f'{data_dir}/outputs'
os.makedirs(outputs_dir, exist_ok=True)
map_path = f'{outputs_dir}/berkeley_housing_map.html'
m.save(map_path)
print(f'✅ Map saved to: {map_path}')

# Display in notebook
m

🗺️ Creating interactive map...
✅ Map saved to: /Users/johngage/berkeley-data/outputs/berkeley_housing_map.html


## 5. Database Export

In [10]:
print('💾 Creating SQLite database...')

# Database path
db_path = f'{data_dir}/berkeley_housing_analysis.db'

# Create database
conn = sqlite3.connect(db_path)

# Export projects table
df.to_sql('projects', conn, if_exists='replace', index=False)

# Create indexes
cursor = conn.cursor()
cursor.execute('CREATE INDEX IF NOT EXISTS idx_year ON projects(year)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_units ON projects(net_units)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_coords ON projects(latitude, longitude)')
conn.commit()

print(f'✅ Database created: {db_path}')

# Test queries
print('\n📊 Testing database queries:')

# Total units by year
query1 = pd.read_sql('''
    SELECT 
        year,
        COUNT(*) as projects,
        SUM(net_units) as total_units
    FROM projects
    GROUP BY year
    ORDER BY year DESC
''', conn)

print('\nUnits by year:')
print(query1.to_string(index=False))

# Largest projects
query2 = pd.read_sql('''
    SELECT 
        address_display,
        net_units,
        year
    FROM projects
    ORDER BY net_units DESC
    LIMIT 10
''', conn)

print('\nTop 10 largest projects:')
print(query2.to_string(index=False))

conn.close()
print('\n✅ Database ready for use')

💾 Creating SQLite database...
✅ Database created: /Users/johngage/berkeley-data/berkeley_housing_analysis.db

📊 Testing database queries:

Units by year:
  year  projects  total_units
2025.0        32        365.0
2024.0        32       3420.0
2023.0         7        388.0
2022.0         8        782.0
2021.0         3         70.0
2020.0         2        258.0

Top 10 largest projects:
    address_display  net_units   year
 1750 SACRAMENTO St      739.0 2024.0
  2276 SHATTUCK Ave      336.0 2024.0
  2700 SHATTUCK Ave      276.0 2024.0
      1914 FIFTH St      257.0 2020.0
    2425 DURANT Ave      250.0 2024.0
2029 UNIVERSITY Ave      240.0 2024.0
 2601 SAN PABLO Ave      223.0 2022.0
  2920 SHATTUCK Ave      221.0 2022.0
     1899 OXFORD St      212.0 2024.0
  3000 SHATTUCK Ave      166.0 2022.0

✅ Database ready for use


## 6. Export Summary Report

In [11]:
print('📄 Generating summary report...')

# Create summary report
report = f'''# Berkeley Housing Development Summary Report 2026 Jan 3

Generated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

## Overview

- **Total Projects:** {len(df)}
- **Total Housing Units:** {df["net_units"].sum():,.0f}
- **Date Range:** {df["year"].min():.0f} - {df["year"].max():.0f}
- **Geocoding Success:** {df["latitude"].notna().sum()}/{len(df)} ({df["latitude"].notna().sum()/len(df)*100:.1f}%)

## Projects by Year

{by_year.to_markdown()}

## Projects by Size

{by_size.to_markdown()}

## Top Streets

{top_streets.head(10).to_markdown()}

## Data Files

- Projects CSV: `housing_projects_final_complete.csv`
- Database: `berkeley_housing_analysis.db`
- Interactive Map: `outputs/berkeley_housing_map.html`

## Notes

- All coordinates validated within Berkeley city bounds
- Data sourced from Berkeley Planning Department permits
- Geocoded using Alameda County GIS address points
'''

report_path = f'{outputs_dir}/ANALYSIS_REPORT.md'
with open(report_path, 'w') as f:
    f.write(report)

print(f'✅ Report saved to: {report_path}')
print('\n' + '='*70)
print('\n🎉 ANALYSIS COMPLETE!')
print('\nCreated files:')
print(f'  • {map_path}')
print(f'  • {db_path}')
print(f'  • {report_path}')

📄 Generating summary report...
✅ Report saved to: /Users/johngage/berkeley-data/outputs/ANALYSIS_REPORT.md


🎉 ANALYSIS COMPLETE!

Created files:
  • /Users/johngage/berkeley-data/outputs/berkeley_housing_map.html
  • /Users/johngage/berkeley-data/berkeley_housing_analysis.db
  • /Users/johngage/berkeley-data/outputs/ANALYSIS_REPORT.md
